# Monte Carlo Simulation including backtesting

## Input

In [36]:
import numpy as np
import pandas as pd
import pandas_datareader
from pandas_datareader import data as wb
import matplotlib.pyplot as plt
from scipy.stats import norm, gmean, cauchy #Package for statistic calculations on distributions
import seaborn as sns  #Package for statistical data visualization
from datetime import datetime
import re

%matplotlib inline

start = '2010-01-01'
end = '2015-01-01' 
forecast = '2016-01-01'
today = datetime.today().strftime('%Y-%m-%d')
   
rf = 0.0078

days = 252
iterations = 10000

benchmark = '^GSPC'
stocks = ['NOVO-B.CO','WM', 'TSLA']

## Monte Carlo Simulation for n assets

### Import Stock Prices from Yahoo Finance and Calculate (log) Returns

Generally, simple returns are used for cross-section perspectives. This is comparing values at one specific point in time (i.e. using calculating Covariance Matrix or Sharpe ratio). 

Log returns are used for statistical evaluation (i.e. MSPE and R-square calculations). Log returns mainly used in time-series. Moreover, stock returns is assumed to have a Log Normal Distribution, therefore Log return is used for statistical evaluation. (comes in handy in analyzing MC simulation as well)

In [37]:
def stock_prices(tickers, start, end):
    prices = pd.DataFrame()
    if len([tickers]) ==1:
        prices[tickers] = wb.DataReader(tickers, data_source = 'yahoo', start = start, end = end)['Adj Close']
        prices = pd.DataFrame(prices)
    else:
        for t in tickers:
            prices[t] = wb.DataReader(t, data_source = 'yahoo', start = start, end = end)['Adj Close']
    return(prices)

def returns(prices):
    return ((prices/prices.shift(1))-1)

def log_returns(prices):
    return (np.log(1+prices.pct_change())) #(prices/prices.shift(1))-1) does not work here. Div0

### Market Data (Benchmark) and Assets Statistics (CAPM, SD, Beta and Sharpe) Calculations

Some calculations to calculate some basic info/statistics such as Expected return from CAPM model. As well as SD, Beta and Sharpe ratio over historical time period. Note: not a main part of the simulation itself

In [38]:
def benchmark_prices_combination(prices, benchmark, start):
    benchmark_prices = stock_prices(benchmark, start, end)
    benchmark_returns = log_returns(benchmark_prices).dropna()
    annual_return = np.exp(benchmark_returns.mean()*252).values-1
    prices = prices.merge(benchmark_prices, left_index=True, right_index=True)
    return prices, annual_return

def calculate_statistics(prices, benchmark = benchmark, start=start, riskfree = rf):
    # Beta
    prices, benchmark_return = benchmark_prices_combination(prices, benchmark, start) #function returns both DF of prices and annual return benchmark. 
    log_return = log_returns(prices)
    cov_matrix = log_return.cov()*252
    covariance = pd.DataFrame(cov_matrix.iloc[:-1,-1]) # finds covariance of all individual assets with benchmark
    market_variance = log_return.iloc[:,-1].var()*252  # finds variance in log returns of benchmark
    beta = covariance/market_variance 
    
    # SD
    sd_assets = pd.DataFrame(((log_return.std()*250**0.5)[:-1]), columns=['STD']) #Calculate SD of asset returns, translate to yearly and skip SD of benchmark.
    beta = beta.merge(sd_assets, left_index=True, right_index=True) # add sd of assets to dataframe of beta.
    
    # CAPM
    for i, row in beta.iterrows(): # loop through panda rows (for all assets)
        beta.at[i,'CAPM'] = riskfree + (row[benchmark] * (benchmark_return-riskfree)) # Calc expected return based on CAPM model.
    
    # Sharpe
    for i, row in beta.iterrows():
        beta.at[i,'Sharpe'] = ((row['CAPM']-riskfree)/(row['STD'])) # Calculate Sharpe of individual assets based on CAPM return. 
    
    beta.rename(columns={benchmark:"Beta"}, inplace=True) #rename column of Beta
    
    return beta #return complete dataframe containing all statistics called beta

### Geometric Brownian Motion
Brownian motion is a stochastic process used for modeling random behavior over time. The price at day t depends on the asset price at day t-1 and the simulated expected return for day t. The motion has two main components:
1. Drift - 
The direction that rates of returns have had in the past. Also know as the historical return of a stock. Assume for now that the historical return of a stock is equal to it's expected return. In the drift calculation, half of the variance is deducted from the expected return. This because historical values are eroded in the future. In other words, if you take the menan of historical returns you "over-estimate" the mean ("expected") return. On the other hand, the mean of the future returns is fixed. 
2. Volatility/SD - 
To account for a error-term in the calculation of future returns we include a random standard normal variable multiplied by the historical standard deviation. At this point we generate random paths in the expected asset prices. For every single time we will simulate the formula (expected asset price at t) we will find a slightly different value.

The asset pricing equation looks as follows:
$$
{Price_{i}}={Price_{i-1} * e^{mean-\frac{1}{2}{Var} + SD * Z(n(0,1))}}
$$

In [39]:
def calculate_drift(prices):
    returns = log_returns(prices)
    mean = returns.mean()
    variance = returns.var()
    drift = mean-(0.5*variance)
    try:
        return drift.values # return array of sd 
    except AttributeError:
        return drift        # return only data type float64

In [40]:
def daily_returns(prices, days, iterations):
    drift = calculate_drift(prices)
    try:
        sd = log_returns(prices).std().values   # returns array of sd (preferred)
    except AttributeError:
        sd = log_returns(prices).std()          # returns only data type float64
    
    random_numbers = np.random.rand(days, iterations) # generated np array of lengts days x iterations
    normal_random = norm.ppf(random_numbers)    # normalize the random numbers generated by a one-tailed test
    daily_returns = np.exp(drift + sd * normal_random) # calc daily returns by formula of GBM
    
    return daily_returns

#### Function to simulate prices
For every t in days (days is the number of days we forecast the stock price) we obtain iterations different options of asset prices at t. In the following function we calculate the asset prices for every day t in days.

In [41]:
def simulate_prices(prices, days, iterations):
    
    returns = daily_returns(prices, days, iterations) # Generate iterations daily returns for days days.
    simulated_prices = np.zeros_like(returns)         # Create empty np array. 
    simulated_prices[0] = prices.iloc[-2]             # Actual price in the first row of matrix. 
    
    for t in range(1,days): # Calculate the price of each day
        simulated_prices[t] = simulated_prices[t-1]*returns[t]  # Fill the matrix by looping through numer of days.
           
    return pd.DataFrame(simulated_prices)

#### Function to collect and summarize random paths of asset prices of all assets in input

In [42]:
def monte_carlo(tickers, days, iterations, start, end):
    prices = stock_prices(tickers, start, end)
    simulated_prices_DF = []
    
    for t in range(len(tickers)): 
        simulated_prices = simulate_prices(prices.iloc[:,t], (days), iterations)        
       
        simulated_prices['ticker'] = tickers[t] # Add column with tickers to np array and append to dataframe.
        column = simulated_prices.columns.tolist()
        column = column[-1:] + column[:-1]
        simulated_prices = simulated_prices[column]
        simulated_prices_DF.append(simulated_prices)
     
    simulated_prices_DF = pd.concat(simulated_prices_DF)
    return simulated_prices_DF

#### Function to calculate simulated (yearly) returns

In [43]:
def simulate_returns(tickers, days, iterations, start, end):
    prices = stock_prices(tickers, start, end)
    actual_prices = list(prices.iloc[-2])
    
    simulated_assets = monte_carlo(tickers, days, iterations, start, end = end)
    average_prices = list(simulated_assets.mean(axis=1))
    average_prices.reverse()
    average_prices = average_prices[::252]
    average_prices.reverse()

    returns = []

    for i in range(len(tickers)):
        return_i = 100 * ((average_prices[i] / actual_prices[i]) -1)
        returns.append(return_i)
    
    returns = [round(return_i, 2) for return_i in returns]
    
    simulated_returns = pd.DataFrame({"Ticker": tickers, "Simulated Return (%)": returns})
    simulated_returns = simulated_returns.sort_values(by = ['Simulated Return (%)'], ascending = False)
    tickers = simulated_returns.iloc[:,0] #arrange ticker list in descending order
    
    return simulated_returns, tickers

#### Function to calculate mean return over historical period and forecasted period

In [44]:
def simulated_mean_prices(tickers, days, iterations, start, end):
    simulated_assets = monte_carlo(tickers, days, iterations, start, end = end)
    #print(simulated_assets)
    simulated_assets["mean"] = simulated_assets.mean(axis=1)
    #print(simulated_assets)
    
    simulated_mean_prices = pd.DataFrame()
    for x in range(len(tickers)):
        begin_a = 0 + days*x
        end_a = days + days*x
        simulated_mean_prices[tickers[x]] = simulated_assets[begin_a:end_a]["mean"] 
        
    return simulated_mean_prices

#### Function to compare forecasted returns with actual returns for day days of forecasted stock prices

Note that this result is not very interesting because it is a statistic of a specific moment in time

In [45]:
def compare_returns(tickers, days, iterations, start, end, forecast):
    simulated_returns, tickers = simulate_returns(tickers, days, iterations, start, end)
    actual_prices = stock_prices(tickers, end, forecast)
    start_prices = list(actual_prices.iloc[1])
    final_prices = list(actual_prices.iloc[-2])

    actual_returns = []

    for i in range(len(tickers)):
        return_i = 100 * ((final_prices[i] - start_prices[i]) / start_prices[i])
        actual_returns.append(return_i)

    actual_returns = [round(return_i, 2) for return_i in actual_returns]
    simulated_returns['Actual Return (%)'] = actual_returns
    compared_returns = simulated_returns
    return compared_returns

#### MAPE (Mean Absolute Percentage Error)
The mean absolute percentage error (MAPE) is a statistical measure of how accurate a forecast system is. It measures this accuracy as a percentage, and can be calculated as the average absolute percent error for each time period minus actual values divided by actual values. Where At is the actual value and Ft is the forecast value, this is given by:

MAPE = $\frac{100\%}{n}\sum_{t=1}^{n}\left |\frac{A_t - F_t}{A_t}\right|$

In [46]:
def mape(actual, predicted): 
    mape = 0
    n=0
    
    for x in range(len(actual)):
        a = abs((actual[x]-predicted[x])/predicted[x])
        if np.isnan(a) == False:
            n += 1
            mape = mape + a
    mape = mape/n

    return mape

#### Function to compare forecasted prices to actual results
Using the correlation and MAPE as measures of prediction accuracy testing the results of the simulation.

In [47]:
def accuracy_results(tickers, days, iterations, start, end, forecast):
    simulated_prices = simulated_mean_prices(tickers, days, iterations, start, end)
    temp_prices = stock_prices(tickers, end, forecast) #Deze heeft de tijd (datums) in de dataframe die zorgen voor problemen met de correlatie
    
    actual_prices = pd.DataFrame()
    for x in range(len(tickers)):
        a = np.array(temp_prices[tickers[x]])
        actual_prices[tickers[x]] = a

    correlations = simulated_prices.corrwith(actual_prices, drop = True)
    
    mape_data = []
    for x in range(len(tickers)):
        m = mape(actual_prices[tickers[x]],simulated_prices[tickers[x]])
        mape_data.append(m)
    
    accuracy_results = pd.DataFrame({"Correlation": correlations, "MAPE": mape_data})
    
    return accuracy_results

#### Function to compare the accuracy results of different intervals of historical periods
In the following function we aim to find the best historical period to use in the MC simulation.

In [50]:
def time_period(tickers, days, iterations, start, end, forecast):
    startingdates = []
    s = int(re.search(r'\d+', start).group()) # First year
    e = int(re.search(r'\d+', end).group()) # Last year

    # Create a list with all possible startingyears
    for a in range(s,e):
        b = start.replace(str(s), str(a))
        startingdates.append(b)
    
    #Statistics for correlation
    minimum = []
    maximum = []
    mean = []
    percentage = []
    enddate = []

    #Statistics for MAPE
    high = []
    good = []
    reasonable = []
    inaccurate = []
    
    for startingdate in startingdates:
        enddate.append(end)
        results = accuracy_results(tickers, days, iterations, start, end, forecast)
        minimum.append(min(results["Correlation"]))
        maximum.append(max(results["Correlation"]))
        mean.append(gmean(results["Correlation"]))
        percentage.append((len([i for i in results["Correlation"] if i > 0.3])/len(results["Correlation"])) )
        high.append((len([i for i in results["MAPE"] if i <= 0.10])/len(results["MAPE"])))
        good.append((len([i for i in results["MAPE"] if i > 0.10 and i <= 0.20])/len(results["MAPE"])))
        reasonable.append((len([i for i in results["MAPE"] if i > 0.20 and i <= 0.50])/len(results["MAPE"])))
        inaccurate.append((len([i for i in results["MAPE"] if i > 0.50])/len(results["MAPE"])))
        print(startingdate)
        print(results)
    
    corr_comparison = pd.DataFrame({"Begindate": startingdates, "Enddate": enddate, "Min": minimum, "Max": maximum, "Mean": mean,"% above 0.75": percentage})
    mape_comparison = pd.DataFrame({"Begindate": startingdates, "Enddate": enddate, "Highly accurate": high, "Good": good, "Reasonable": reasonable, "Inaccurate": inaccurate})
    
    
    return corr_comparison, mape_comparison

In [53]:
time_period(stocks, 257, 10000, start, end, forecast)

2010-01-01
           Correlation      MAPE
NOVO-B.CO     0.675513  0.212288
WM            0.177494  0.056490
TSLA          0.349103  0.193922
2011-01-01
           Correlation      MAPE
NOVO-B.CO     0.673784  0.213338
WM            0.185961  0.054816
TSLA          0.365125  0.188643
2012-01-01
           Correlation      MAPE
NOVO-B.CO     0.673991  0.211774
WM            0.175058  0.058623
TSLA          0.365994  0.189958
2013-01-01
           Correlation      MAPE
NOVO-B.CO     0.678466  0.209388
WM            0.194858  0.055611
TSLA          0.345776  0.186607
2014-01-01
           Correlation      MAPE
NOVO-B.CO     0.675219  0.210918
WM            0.186053  0.055656
TSLA          0.356583  0.189993


(    Begindate     Enddate       Min       Max      Mean  % above 0.75
 0  2010-01-01  2015-01-01  0.177494  0.675513  0.347208      0.666667
 1  2011-01-01  2015-01-01  0.185961  0.673784  0.357653      0.666667
 2  2012-01-01  2015-01-01  0.175058  0.673991  0.350835      0.666667
 3  2013-01-01  2015-01-01  0.194858  0.678466  0.357559      0.666667
 4  2014-01-01  2015-01-01  0.186053  0.675219  0.355151      0.666667,
     Begindate     Enddate  Highly accurate      Good  Reasonable  Inaccurate
 0  2010-01-01  2015-01-01         0.333333  0.333333    0.333333         0.0
 1  2011-01-01  2015-01-01         0.333333  0.333333    0.333333         0.0
 2  2012-01-01  2015-01-01         0.333333  0.333333    0.333333         0.0
 3  2013-01-01  2015-01-01         0.333333  0.333333    0.333333         0.0
 4  2014-01-01  2015-01-01         0.333333  0.333333    0.333333         0.0)